<a href="https://colab.research.google.com/github/sankethiyer217/Speech_Recognition/blob/main/11_09_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'
!kaggle datasets download -d mfekadu/darpa-timit-acousticphonetic-continuous-speech

 99% 818M/829M [00:03<00:00, 241MB/s]
100% 829M/829M [00:03<00:00, 260MB/s]


In [2]:
!unzip -q *.zip

In [3]:
import tensorflow as tf
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import re
import os

In [4]:
imp_phoneme = {
    'iy':'iy',  'ih':'ih',   'eh':'eh',  'ae':'ae',    'ix':'ih',  'ax':'ah',   'ah':'ah',  'uw':'uw',
    'ux':'uw',  'uh':'uh',   'ao':'aa',  'aa':'aa',    'ey':'ey',  'ay':'ay',   'oy':'oy',  'aw':'aw',
    'ow':'ow',  'l':'l',     'el':'l',  'r':'r',      'y':'y',    'w':'w',     'er':'er',  'axr':'er',
    'm':'m',    'em':'m',     'n':'n',    'nx':'n',     'en':'n',  'ng':'ng',   'eng':'ng', 'ch':'ch',
    'jh':'jh',  'dh':'dh',   'b':'b',    'd':'d',      'dx':'dx',  'g':'g',     'p':'p',    't':'t',
    'k':'k',    'z':'z',     'zh':'sh',  'v':'v',      'f':'f',    'th':'th',   's':'s',    'sh':'sh',
    'hh':'hh',  'hv':'hh',   'pcl':'h#', 'tcl':'h#', 'kcl':'h#', 'qcl':'h#','bcl':'h#','dcl':'h#',
    'gcl':'h#','h#':'h#',  '#h':'h#',  'pau':'h#', 'epi': 'h#','nx':'n',   'ax-h':'ah','q':'h#' 
}

In [5]:
def not_phoneme(x):
    try:
        return imp_phoneme[x]
    except:
        return 'h#'

In [6]:
phon39 = list(set(imp_phoneme.values()))

label_p39 = {}
for i, p in enumerate(phon39):
    label_p39[p] = i+1

In [7]:
from tqdm import tqdm

In [ ]:
acoustic_features = []
label_vectors = []
for dirname, _, filenames in tqdm(os.walk('/content/data/TRAIN/')):
    for filename in filenames:
        wav_filepath = os.path.join(dirname, filename)
        if '.wav' not in wav_filepath:
          if '.WAV' in wav_filepath:

              ph_filepath = wav_filepath.replace('.WAV', '.PHN')
              ph_df = pd.read_csv(ph_filepath, sep=' ', header=None)
              ph_df.columns = ['start', 'end', 'phoneme']
              ph_df['phoneme'] = ph_df['phoneme'].apply(lambda x: re.sub("[^a-zA-Z]","",x))
              ph_df['phoneme'] = ph_df['phoneme'].apply(lambda x: not_phoneme(x))

              x , sr = librosa.load(wav_filepath)
              mfcc = librosa.feature.mfcc(x, sr=sr, n_mfcc=128).T
              db_mfcc = librosa.amplitude_to_db(mfcc, ref=np.max)
              db_mfcc = np.mean(mfcc, axis=0)
              mel_spec=librosa.feature.melspectrogram(x, sr=sr, n_fft=2048).T
              db_mel=librosa.amplitude_to_db(mel_spec, ref=np.max)
              db_mel = np.mean(mel_spec, axis=0)

              label = [0 for i in range(39)]
              for l in ph_df['phoneme'].unique():
                  label[label_p39[l]-1] = 1
              label_vectors.append(label)
              acoustic_features.append(db_mfcc+db_mel)

351it [06:57,  1.29s/it]

In [9]:
acoustic_features_ = np.array(acoustic_features)
label_vectors_ = np.array(label_vectors)

In [10]:
acoustic_features_.shape

(4620, 128)

In [11]:
training_size = int(acoustic_features_.shape[0] * 0.80)
training_size

3696

In [12]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(128,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(39, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               66048     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 39)                5031      
                                                                 
Total params: 235,303
Trainable params: 235,303
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(acoustic_features_[:training_size], label_vectors_[:training_size],
                    epochs=200,
                    validation_data=(acoustic_features_[training_size:], label_vectors_[training_size:]))

Epoch 1/200
116/116 [==============================] - 0s 3ms/step - loss: 34969407488.0000 - accuracy: 0.0387 - val_loss: 34776145920.0000 - val_accuracy: 0.0000e+00
Epoch 2/200
116/116 [==============================] - 0s 3ms/step - loss: 39447117824.0000 - accuracy: 0.0354 - val_loss: 39974383616.0000 - val_accuracy: 0.0000e+00
Epoch 3/200
116/116 [==============================] - 0s 3ms/step - loss: 43648397312.0000 - accuracy: 0.0330 - val_loss: 43653111808.0000 - val_accuracy: 0.4545
Epoch 4/200
116/116 [==============================] - 0s 3ms/step - loss: 50416996352.0000 - accuracy: 0.0344 - val_loss: 50256179200.0000 - val_accuracy: 0.0000e+00
Epoch 5/200
116/116 [==============================] - 0s 3ms/step - loss: 55000788992.0000 - accuracy: 0.0379 - val_loss: 55409954816.0000 - val_accuracy: 0.0000e+00
Epoch 6/200
116/116 [==============================] - 0s 3ms/step - loss: 60897738752.0000 - accuracy: 0.0308 - val_loss: 61777027072.0000 - val_accuracy: 0.0000e+00
Ep